In [1]:
import sys
import json

from pathlib import Path
from dateutil import parser

import requests
import fiona

import pandas as pd
import geopandas as gpd

DIR = Path('..')
sys.path.append(str(DIR))

import pyotp as otp

DATA_DIR = DIR/'data/'

%load_ext autoreload
%autoreload 2

# Palyground

In [24]:
# setting the analysis date and time
dt = parser.parse('11/Jan/2018 8:00:00 AM')

in_gdf = gpd.read_file(str(DATA_DIR/'sample_locations.geojson'))

In [25]:
in_gdf

,stop_id,stop_code,stop_name,stop_lat,stop_lon,geometry
0,1321.0,7427.0,659 Remuera Rd,-36.879865,174.81715,POINT (174.8171499998194 -36.87986499959732)
1,5107.0,7083.0,Mayoral Drive Opposite Aotea Centre,-36.851833,174.76099,POINT (174.760989999634 -36.85183300038418)
2,1843.0,8314.0,464 Sandringham Rd,-36.888577,174.73741,POINT (174.7374099997028 -36.88857699965718)


In [63]:
otp.route(
    locations_gdf = in_gdf.head(2), #a pair of locations in geodataframe fromat
    mode = 'TRANSIT,WALK',
    trip_name = 'test', 
    date_time = dt,
    control_vars = {
        'maxWalkDistance':'500',
        'wheelchair':False,
    },
)

,trip_name,leg_id,mode,from,from_name,to,to_name,route_id,trip_id,distance,duration,startTime,endTime,waitTime,geometry
0,test,0,WALK,POINT (174.8171499998194 -36.87986499959732),Origin,POINT (174.81715 -36.87986),1:7427-20171218162726_v61.9,,,0.192,1,1515611021000,1515611022000,1000,"LINESTRING (17.481712 -3.687981, 17.481712 -3...."
1,test,1,BUS,POINT (174.81715 -36.87986),1:7427-20171218162726_v61.9,POINT (174.76449 -36.85741),1:7149-20171218162726_v61.9,1:62501-20171218162726_v61.9,1:14625085069-20171218162726_v61.9,6028.86,1957,1515611023000,1515612980000,0,"LINESTRING (17.481715 -3.687986, 17.481712 -3...."
2,test,2,WALK,POINT (174.76449 -36.85741),1:7149-20171218162726_v61.9,POINT (174.76216 -36.85603),1:7061-20171218162726_v61.9,,,335.519,255,1515612980000,1515613235000,188000,"LINESTRING (17.476455 -3.685747, 17.476472 -3...."
3,test,3,BUS,POINT (174.76216 -36.85603),1:7061-20171218162726_v61.9,POINT (174.76099 -36.85183),1:7083-20171218162726_v61.9,1:13403-20171218162726_v61.9,1:1065107432-20171218162726_v61.9,483.136,170,1515613423000,1515613593000,1000,"LINESTRING (17.476216 -3.685603, 17.476224 -3...."
4,test,4,WALK,POINT (174.76099 -36.85183),1:7083-20171218162726_v61.9,POINT (174.760989999634 -36.85183300038418),Destination,,,0.242,1,1515613594000,1515613595000,NaN,"LINESTRING (17.476103 -3.685187, 17.476102 -3...."


In [27]:
otp.service_area(
    in_gdf = in_gdf,
    id_field = 'stop_name',
    mode = "TRANSIT,WALK", 
    breaks = [500, 1000], #in seconds
    date_time = dt,
    control_vars = {'maxWalkDistance':'400'},
)

,geometry,time,name
0,"(POLYGON ((174.7889 -36.8799, 174.7894 -36.880...",1000,659 Remuera Rd
1,"(POLYGON ((174.8096 -36.8799, 174.8082 -36.881...",500,659 Remuera Rd
0,"(POLYGON ((174.761 -36.8643, 174.7631 -36.8626...",1000,Mayoral Drive Opposite Aotea Centre
1,"(POLYGON ((174.761 -36.855, 174.7616 -36.8549,...",500,Mayoral Drive Opposite Aotea Centre
0,"(POLYGON ((174.7374 -36.8762, 174.7354 -36.877...",1000,464 Sandringham Rd
1,"(POLYGON ((174.7374 -36.8844, 174.736 -36.885,...",500,464 Sandringham Rd


In [28]:
od_set = gpd.read_file(str(DATA_DIR/'od_set.geojson'))
od_set['location_name'] = 'location: '+ od_set.index.astype(str)
od_set

,route_id,geometry,location_name
0,1,POINT (174.7298018990001 -36.87621789599996),location: 0
1,1,POINT (174.7639865660001 -36.84994778099997),location: 1
2,2,POINT (174.7647225510001 -36.90041207899998),location: 2
3,2,POINT (174.7271483520001 -36.89949745399997),location: 3
4,3,POINT (174.7085681680001 -36.88324049999994),location: 4
5,3,POINT (174.8049270710001 -36.9133301599999),location: 5
6,4,POINT (174.830714979 -36.89881761199996),location: 6
7,4,POINT (174.835486799 -36.86993589899998),location: 7
8,5,POINT (174.7980088750001 -36.88176517999995),location: 8
9,5,POINT (174.7413818690001 -36.85628036899993),location: 9


In [29]:
otp.od_matrix_unlimited(
    origins = od_set,
    destinations = od_set,
    mode = 'multimodal',
    origins_name = 'location_name',
    destinations_name = 'location_name',
    date_time = dt,
    control_vars = {'maxWalkDistance':'400'},
)

Analysis started at: 2018-04-12 14:54:16.596202
Total routes caclulated: 50, time: 0:00:07.340572
Total routes caclulated: 100, time: 0:00:19.941563


,trip_name,leg_id,mode,from,from_name,to,to_name,route_id,trip_id,distance,duration,startTime,endTime,waitTime,geometry
0,from location: 0 to location: 1,0,WALK,POINT (174.7298018990001 -36.87621789599996),Origin,POINT (174.73232 -36.8756),1:8219-20171218162726_v61.9,,,283.621,224,1515611057000,1515611281000,1000,"LINESTRING (17.47298 -3.687612, 17.472991 -3.6..."
1,from location: 0 to location: 1,1,BUS,POINT (174.73232 -36.8756),1:8219-20171218162726_v61.9,POINT (174.76456 -36.8489),1:7096-20171218162726_v61.9,1:20903-20171218162726_v61.9,1:1100107339-20171218162726_v61.9,4659.82,1318,1515611282000,1515612600000,1000,"LINESTRING (17.473232 -3.68756, 17.473236 -3.6..."
2,from location: 0 to location: 1,2,WALK,POINT (174.76456 -36.8489),1:7096-20171218162726_v61.9,POINT (174.7639865660001 -36.84994778099997),Destination,,,150.682,129,1515612601000,1515612730000,NaN,"LINESTRING (17.476456 -3.684889, 17.476441 -3...."
3,from location: 0 to location: 2,0,WALK,POINT (174.7298018990001 -36.87621789599996),Origin,POINT (174.73232 -36.8756),1:8219-20171218162726_v61.9,,,283.621,224,1515611040000,1515611264000,1000,"LINESTRING (17.47298 -3.687612, 17.472991 -3.6..."
4,from location: 0 to location: 2,1,BUS,POINT (174.73232 -36.8756),1:8219-20171218162726_v61.9,POINT (174.73479 -36.87469),1:8217-20171218162726_v61.9,1:22107-20171218162726_v61.9,1:18157107386-20171218162726_v61.9,241.888,78,1515611265000,1515611343000,0,"LINESTRING (17.473232 -3.68756, 17.473236 -3.6..."
5,from location: 0 to location: 2,2,WALK,POINT (174.73479 -36.87469),1:8217-20171218162726_v61.9,POINT (174.7374 -36.87372),1:8214-20171218162726_v61.9,,,263.274,200,1515611343000,1515611543000,291000,"LINESTRING (17.473483 -3.687473, 17.473492 -3...."
6,from location: 0 to location: 2,3,BUS,POINT (174.7374 -36.87372),1:8214-20171218162726_v61.9,POINT (174.71584 -36.88404),1:8230-20171218162726_v61.9,1:22308-20171218162726_v61.9,1:13223107453-20171218162726_v61.9,2682.18,525,1515611834000,1515612359000,0,"LINESTRING (17.47374 -3.687372, 17.473739 -3.6..."
7,from location: 0 to location: 2,4,WALK,POINT (174.71584 -36.88404),1:8230-20171218162726_v61.9,POINT (174.71663 -36.88473),1:8812-20171218162726_v61.9,,,111.84,87,1515612359000,1515612446000,154000,"LINESTRING (17.471592 -3.68841, 17.471586 -3.6..."
8,from location: 0 to location: 2,5,BUS,POINT (174.71663 -36.88473),1:8812-20171218162726_v61.9,POINT (174.76394 -36.90947),1:8844-20171218162726_v61.9,1:01002-20171218162726_v61.9,1:14010107475-20171218162726_v61.9,5299.73,1008,1515612600000,1515613608000,0,"LINESTRING (17.471663 -3.688473, 17.471658 -3...."
9,from location: 0 to location: 2,6,WALK,POINT (174.76394 -36.90947),1:8844-20171218162726_v61.9,POINT (174.76361 -36.90893),1:8649-20171218162726_v61.9,,,93.42,73,1515613608000,1515613681000,3000,"LINESTRING (17.476392 -3.690951, 17.476355 -3...."
